In [1]:
import pandas as pd
import numpy as np
import csv
import sqlite3
import re
import pickle

from sklearn.datasets import load_wine
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import SVC
from sklearn.metrics import roc_curve, auc

from sklearn.metrics import accuracy_score

import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
csv_path = './data/loan.csv'
date_to_parse = ['issue_d' , 'last_pymnt_d', 'next_pymnt_d', 'last_credit_pull_d', 'earliest_cr_line']

df=pd.read_csv(csv_path, date_parser=date_to_parse)

C:\Users\xzhou\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (19,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
df.shape

(887379, 74)

In [6]:
df.columns

Index(['id', 'member_id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv',
       'term', 'int_rate', 'installment', 'grade', 'sub_grade', 'emp_title',
       'emp_length', 'home_ownership', 'annual_inc', 'verification_status',
       'issue_d', 'loan_status', 'pymnt_plan', 'url', 'desc', 'purpose',
       'title', 'zip_code', 'addr_state', 'dti', 'delinq_2yrs',
       'earliest_cr_line', 'inq_last_6mths', 'mths_since_last_delinq',
       'mths_since_last_record', 'open_acc', 'pub_rec', 'revol_bal',
       'revol_util', 'total_acc', 'initial_list_status', 'out_prncp',
       'out_prncp_inv', 'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp',
       'total_rec_int', 'total_rec_late_fee', 'recoveries',
       'collection_recovery_fee', 'last_pymnt_d', 'last_pymnt_amnt',
       'next_pymnt_d', 'last_credit_pull_d', 'collections_12_mths_ex_med',
       'mths_since_last_major_derog', 'policy_code', 'application_type',
       'annual_inc_joint', 'dti_joint', 'verification_status_joint',
    

In [7]:
filter = ['loan_amnt', 'term', 'int_rate', 'installment', 'grade', 
          'emp_length', 'home_ownership', 'annual_inc', 'loan_status', 
          'purpose', 'dti', 'delinq_2yrs', 'earliest_cr_line', 
          'inq_last_6mths', 'mths_since_last_delinq', 
          'mths_since_last_record', 'open_acc', 'pub_rec', 'revol_bal', 
          'revol_util', 'application_type', 'collections_12_mths_ex_med', 
          'mths_since_last_major_derog', 'policy_code', 'annual_inc_joint', 
          'dti_joint', 'acc_now_delinq', 'tot_coll_amt', 'tot_cur_bal', 
          'open_acc_6m', 'open_il_6m', 'open_il_12m', 'open_il_24m', 
          'mths_since_rcnt_il', 'total_bal_il', 'il_util', 'open_rv_12m', 
          'open_rv_24m', 'max_bal_bc', 'all_util', 'total_rev_hi_lim', 
          'inq_fi', 'total_cu_tl','inq_last_12m']

In [8]:
df_smaller = df.filter(items=filter)

In [9]:
df_smaller.columns

Index(['loan_amnt', 'term', 'int_rate', 'installment', 'grade', 'emp_length',
       'home_ownership', 'annual_inc', 'loan_status', 'purpose', 'dti',
       'delinq_2yrs', 'earliest_cr_line', 'inq_last_6mths',
       'mths_since_last_delinq', 'mths_since_last_record', 'open_acc',
       'pub_rec', 'revol_bal', 'revol_util', 'application_type',
       'collections_12_mths_ex_med', 'mths_since_last_major_derog',
       'policy_code', 'annual_inc_joint', 'dti_joint', 'acc_now_delinq',
       'tot_coll_amt', 'tot_cur_bal', 'open_acc_6m', 'open_il_6m',
       'open_il_12m', 'open_il_24m', 'mths_since_rcnt_il', 'total_bal_il',
       'il_util', 'open_rv_12m', 'open_rv_24m', 'max_bal_bc', 'all_util',
       'total_rev_hi_lim', 'inq_fi', 'total_cu_tl', 'inq_last_12m'],
      dtype='object')

In [10]:
df_smaller.isnull().sum()

loan_amnt                           0
term                                0
int_rate                            0
installment                         0
grade                               0
emp_length                      44825
home_ownership                      0
annual_inc                          4
loan_status                         0
purpose                             0
dti                                 0
delinq_2yrs                        29
earliest_cr_line                   29
inq_last_6mths                     29
mths_since_last_delinq         454312
mths_since_last_record         750326
open_acc                           29
pub_rec                            29
revol_bal                           0
revol_util                        502
application_type                    0
collections_12_mths_ex_med        145
mths_since_last_major_derog    665676
policy_code                         0
annual_inc_joint               886868
dti_joint                      886870
acc_now_deli

In [11]:
# Drop the filds have 90%+ null values, 
# as filling in the nulls can skew the model

columns_to_drop = [
    'annual_inc_joint', 'dti_joint',  'open_acc_6m', 'open_il_6m',
    'open_il_12m', 'open_il_24m', 'mths_since_rcnt_il', 'total_bal_il',
    'il_util', 'open_rv_12m', 'open_rv_24m', 'max_bal_bc', 'all_util',
    'inq_fi', 'total_cu_tl', 'inq_last_12m'
]

df_smaller.drop(columns_to_drop, axis=1, inplace=True)

In [12]:
df_smaller.rename(index=str, 
                  columns={"dti": "payment_to_income_ratio_individual"}, 
                  inplace=True)

In [13]:
df_clean = df_smaller.copy()

In [14]:
df_clean.isnull().sum()

loan_amnt                                  0
term                                       0
int_rate                                   0
installment                                0
grade                                      0
emp_length                             44825
home_ownership                             0
annual_inc                                 4
loan_status                                0
purpose                                    0
payment_to_income_ratio_individual         0
delinq_2yrs                               29
earliest_cr_line                          29
inq_last_6mths                            29
mths_since_last_delinq                454312
mths_since_last_record                750326
open_acc                                  29
pub_rec                                   29
revol_bal                                  0
revol_util                               502
application_type                           0
collections_12_mths_ex_med               145
mths_since

In [15]:
excluded = {'Issued', 'Does not meet the credit policy. Status:Charged Off',
            'Does not meet the credit policy. Status:Fully Paid'}
good = {'Current', 'Fully Paid'}
bad = {'Charged Off', 'Default', 'In Grace Period', 
       'Late (16-30 days)', 'Late (31-120 days)'}

def get_status(status):
    """
    Classify loans to either good or bad loans based on their status.
    """
    try: 
        status=status.strip()
        if status in good:
            return "good"
        elif status in bad:
            return "bad"
        else: 
            return "excluded"
    except: 
        return "excluded"

In [16]:
df_clean['loan_status'] = df_clean['loan_status'].apply(lambda x: get_status(x))

In [17]:
df_clean = df_clean[df_clean['loan_status'] != 'excluded']

In [18]:
df_clean.groupby('loan_status').count()

,loan_amnt,term,int_rate,installment,grade,emp_length,home_ownership,annual_inc,purpose,payment_to_income_ratio_individual,...,revol_bal,revol_util,application_type,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,acc_now_delinq,tot_coll_amt,tot_cur_bal,total_rev_hi_lim
loan_status,,,,,,,,,,,,,,,,,,,,,
bad,66668,66668,66668,66668,66668,62973,66668,66668,66668,66668,...,66668,66607,66668,66662,15124,66668,66668,56417,56417,56417
good,809502,809502,809502,809502,809502,769037,809502,809502,809502,809502,...,809502,809104,809502,809452,204054,809502,809502,752226,752226,752226


In [19]:
# Fill the null value for 'mths_since_last_delinq' with zero as zero is default case for majority of loans

fill_with_zero = {
    'mths_since_last_delinq':0,
    'mths_since_last_record':0,
    'collections_12_mths_ex_med':0,
    'mths_since_last_major_derog':0,
    'tot_coll_amt':0
}
            
df_clean.fillna(fill_with_zero,inplace=True)

In [20]:
# Fill the null value for 'revol util' with means

df_clean['revol_util'].fillna(
    value=df_clean['revol_util'].mean(), inplace=True)

In [21]:
# Fill the null value for 'tot_cur_bal' with means

df_clean['tot_cur_bal'].fillna(
    value=df_clean['tot_cur_bal'].mean(), inplace=True)

In [22]:
# Fill the null value for 'total_rev_hi_lim' with means

df_clean['total_rev_hi_lim'].fillna(
    value=df_clean['total_rev_hi_lim'].mean(), inplace=True)

In [25]:
def get_employment_length(length):
    """
    This is to extract numerical value of emp_length
    For simplicity, approximation was used.
    <1 years will be treated as 1 year
    >10 years will be treated as 10 years
    """
    try:
        length = length.strip()
        year = re.findall(r'(\d+)', length)
        return int(year[0]) 
    except:
        return None

In [26]:
df_clean['emp_length'] = df_clean['emp_length'].apply(lambda x: get_employment_length(x))

In [27]:
df_clean['emp_length'].fillna(
    value=df_clean['emp_length'].mean(), inplace=True)

In [28]:
# All Null values have been appropriately handled.

df_clean.isnull().sum()

loan_amnt                             0
term                                  0
int_rate                              0
installment                           0
grade                                 0
emp_length                            0
home_ownership                        0
annual_inc                            0
loan_status                           0
purpose                               0
payment_to_income_ratio_individual    0
delinq_2yrs                           0
earliest_cr_line                      0
inq_last_6mths                        0
mths_since_last_delinq                0
mths_since_last_record                0
open_acc                              0
pub_rec                               0
revol_bal                             0
revol_util                            0
application_type                      0
collections_12_mths_ex_med            0
mths_since_last_major_derog           0
policy_code                           0
acc_now_delinq                        0


In [29]:
# Break down terms of loan into 2 categories.

df_clean = pd.get_dummies(df_clean, prefix=['term'], columns=['term'])

In [30]:
# Break down grades into categories.

df_clean = pd.get_dummies(df_clean, prefix=['grade'], columns=['grade'])

In [31]:
df_clean = pd.get_dummies(df_clean, prefix=['home_ownership'], columns=['home_ownership'])

In [32]:
df_clean[df_clean['loan_status'] == 'bad'].groupby('purpose').count()

,loan_amnt,int_rate,installment,emp_length,annual_inc,loan_status,payment_to_income_ratio_individual,delinq_2yrs,earliest_cr_line,inq_last_6mths,...,grade_D,grade_E,grade_F,grade_G,home_ownership_ANY,home_ownership_MORTGAGE,home_ownership_NONE,home_ownership_OTHER,home_ownership_OWN,home_ownership_RENT
purpose,,,,,,,,,,,,,,,,,,,,,
car,583,583,583,583,583,583,583,583,583,583,...,583,583,583,583,583,583,583,583,583,583
credit_card,11686,11686,11686,11686,11686,11686,11686,11686,11686,11686,...,11686,11686,11686,11686,11686,11686,11686,11686,11686,11686
debt_consolidation,41316,41316,41316,41316,41316,41316,41316,41316,41316,41316,...,41316,41316,41316,41316,41316,41316,41316,41316,41316,41316
educational,56,56,56,56,56,56,56,56,56,56,...,56,56,56,56,56,56,56,56,56,56
home_improvement,3482,3482,3482,3482,3482,3482,3482,3482,3482,3482,...,3482,3482,3482,3482,3482,3482,3482,3482,3482,3482
house,406,406,406,406,406,406,406,406,406,406,...,406,406,406,406,406,406,406,406,406,406
major_purchase,1271,1271,1271,1271,1271,1271,1271,1271,1271,1271,...,1271,1271,1271,1271,1271,1271,1271,1271,1271,1271
medical,782,782,782,782,782,782,782,782,782,782,...,782,782,782,782,782,782,782,782,782,782
moving,592,592,592,592,592,592,592,592,592,592,...,592,592,592,592,592,592,592,592,592,592


In [33]:
df_clean = pd.get_dummies(df_clean, prefix=['purpose'], columns=['purpose'])

In [34]:
def get_earliest_cr_line(string):
    """
    This is to calculate years of credit history.
    Since the data is from 2016, use 2016 minus the 
    starting year to estimate years of credit history.
    
    """
    string = string.strip()
    year = re.findall(r'(\d+)', string)
    return (2016-int(year[0])) 

In [35]:
df_clean['earliest_cr_line'] = df_clean['earliest_cr_line'].apply(lambda x: get_earliest_cr_line(x))

In [36]:
def inverse(value):
    try:
        return 1 / value
    except: 
        return 0

In [37]:
# Take inverse of months since last delinquishing date as this is is inversed related to default prediction

df_clean['inversed_mths_since_last_delinq'] = df_clean['mths_since_last_delinq'].apply(lambda x: inverse(x))

In [38]:
# Take inverse of months since last record as this is is inversed related to default to default prediction

df_clean['inversed_mths_since_last_record'] = df_clean['mths_since_last_record'].apply(lambda x: inverse(x))

In [39]:
# Take inverse of months since last record as this is is inversed related to default to default prediction.
new_col_name = 'inversed_mths_since_last_major_derog'
old_col_name = 'mths_since_last_major_derog'
df_clean[new_col_name] = df_clean[old_col_name].apply(lambda x: inverse(x))

In [40]:
df_clean.drop(
    ['mths_since_last_delinq', 'mths_since_last_record',
     'mths_since_last_major_derog'], 
    inplace=True, axis=1)

In [41]:
# Calculate income-to-debt raio. 
# This can result in infinite values when revol_balance is small.

df_clean['income_to_debt_ratio'] = df_clean['annual_inc'] / df_clean['revol_bal']

In [42]:
# Replace infinite value with N/As.

df_clean['income_to_debt_ratio'] = df_clean['income_to_debt_ratio'].replace(np.inf, np.nan)

In [43]:
# Drop infinite values since size of it is very small.

df_clean = df_clean[df_clean['income_to_debt_ratio'] >= 0]

In [44]:
# Break down applicaiton type to different categories.

df_clean=pd.get_dummies(df_clean, prefix=['application_type'], columns=['application_type'])

In [45]:
df_clean.columns

Index(['loan_amnt', 'int_rate', 'installment', 'emp_length', 'annual_inc',
       'loan_status', 'payment_to_income_ratio_individual', 'delinq_2yrs',
       'earliest_cr_line', 'inq_last_6mths', 'open_acc', 'pub_rec',
       'revol_bal', 'revol_util', 'collections_12_mths_ex_med', 'policy_code',
       'acc_now_delinq', 'tot_coll_amt', 'tot_cur_bal', 'total_rev_hi_lim',
       'term_ 36 months', 'term_ 60 months', 'grade_A', 'grade_B', 'grade_C',
       'grade_D', 'grade_E', 'grade_F', 'grade_G', 'home_ownership_ANY',
       'home_ownership_MORTGAGE', 'home_ownership_NONE',
       'home_ownership_OTHER', 'home_ownership_OWN', 'home_ownership_RENT',
       'purpose_car', 'purpose_credit_card', 'purpose_debt_consolidation',
       'purpose_educational', 'purpose_home_improvement', 'purpose_house',
       'purpose_major_purchase', 'purpose_medical', 'purpose_moving',
       'purpose_other', 'purpose_renewable_energy', 'purpose_small_business',
       'purpose_vacation', 'purpose_wedding',

In [46]:
df_clean['loan_status'] = (df_clean['loan_status']=='bad').astype(int)

In [47]:
df_clean.head(20)

,loan_amnt,int_rate,installment,emp_length,annual_inc,loan_status,payment_to_income_ratio_individual,delinq_2yrs,earliest_cr_line,inq_last_6mths,...,purpose_renewable_energy,purpose_small_business,purpose_vacation,purpose_wedding,inversed_mths_since_last_delinq,inversed_mths_since_last_record,inversed_mths_since_last_major_derog,income_to_debt_ratio,application_type_INDIVIDUAL,application_type_JOINT
0,5000.0,10.65,162.87,10.0,24000.00,0,27.65,0.0,31,1.0,...,0,0,0,0,0.000000,0.0,0.0,1.758499,1,0
1,2500.0,15.27,59.83,1.0,30000.00,1,1.00,0.0,17,5.0,...,0,0,0,0,0.000000,0.0,0.0,17.783047,1,0
2,2400.0,15.96,84.33,10.0,12252.00,0,8.72,0.0,15,2.0,...,0,1,0,0,0.000000,0.0,0.0,4.144790,1,0
3,10000.0,13.49,339.31,10.0,49200.00,0,20.00,0.0,20,1.0,...,0,0,0,0,0.028571,0.0,0.0,8.788853,1,0
4,3000.0,12.69,67.79,1.0,80000.00,0,17.94,0.0,20,0.0,...,0,0,0,0,0.026316,0.0,0.0,2.879459,1,0
5,5000.0,7.90,156.46,3.0,36000.00,0,11.20,0.0,12,3.0,...,0,0,0,1,0.000000,0.0,0.0,4.520909,1,0
6,7000.0,15.96,170.08,8.0,47004.00,0,23.51,0.0,11,1.0,...,0,0,0,0,0.000000,0.0,0.0,2.651698,1,0
7,3000.0,18.64,109.43,9.0,48000.00,0,5.35,0.0,9,2.0,...,0,0,0,0,0.000000,0.0,0.0,5.838706,1,0
8,5600.0,21.28,152.39,4.0,40000.00,1,5.55,0.0,12,2.0,...,0,1,0,0,0.000000,0.0,0.0,7.677543,1,0
9,5375.0,12.69,121.45,1.0,15000.00,1,18.08,0.0,12,0.0,...,0,0,0,0,0.000000,0.0,0.0,1.616554,1,0


In [48]:
df_clean.corr()

,loan_amnt,int_rate,installment,emp_length,annual_inc,loan_status,payment_to_income_ratio_individual,delinq_2yrs,earliest_cr_line,inq_last_6mths,...,purpose_renewable_energy,purpose_small_business,purpose_vacation,purpose_wedding,inversed_mths_since_last_delinq,inversed_mths_since_last_record,inversed_mths_since_last_major_derog,income_to_debt_ratio,application_type_INDIVIDUAL,application_type_JOINT
loan_amnt,1.000000,0.145413,0.944812,0.103315,0.332569,0.005463,0.019483,0.000317,0.169350,-0.028376,...,-0.014173,0.008769,-0.073101,-0.025564,0.014590,-0.040086,-0.017115,-0.003100,-0.011012,0.011012
int_rate,0.145413,1.000000,0.133254,0.009411,-0.072979,0.176271,0.079449,0.055138,-0.104494,0.232718,...,0.012860,0.074271,0.018685,0.010759,0.051977,0.026934,0.042863,-0.004582,-0.009636,0.009636
installment,0.944812,0.133254,1.000000,0.087992,0.325900,0.016039,0.013205,0.008330,0.150521,0.002401,...,-0.012090,0.022200,-0.069483,-0.021585,0.020735,-0.033768,-0.010531,-0.002656,-0.009004,0.009004
emp_length,0.103315,0.009411,0.087992,1.000000,0.062809,-0.015148,0.019743,0.024928,0.207999,-0.006142,...,0.000433,-0.016398,0.004387,-0.022119,0.031750,0.014264,0.019594,-0.004619,-0.000534,0.000534
annual_inc,0.332569,-0.072979,0.325900,0.062809,1.000000,-0.034007,-0.087226,0.047715,0.143974,0.037144,...,-0.000428,0.024453,-0.008886,-0.004412,0.048312,0.006686,0.017940,0.020773,0.006152,-0.006152
loan_status,0.005463,0.176271,0.016039,-0.015148,-0.034007,1.000000,0.010962,0.003177,-0.018992,0.073945,...,0.004448,0.037346,0.002349,0.008742,0.000458,-0.009653,-0.004193,0.001053,0.005535,-0.005535
payment_to_income_ratio_individual,0.019483,0.079449,0.013205,0.019743,-0.087226,0.010962,1.000000,-0.002928,0.013093,-0.005728,...,-0.003371,-0.025280,-0.004105,-0.011941,-0.000443,-0.014612,-0.007076,-0.013035,-0.077614,0.077614
delinq_2yrs,0.000317,0.055138,0.008330,0.024928,0.047715,0.003177,-0.002928,1.000000,0.082433,0.023656,...,-0.001585,0.001766,0.000763,-0.004711,0.462281,0.010064,0.294611,0.008382,-0.000858,0.000858
earliest_cr_line,0.169350,-0.104494,0.150521,0.207999,0.143974,-0.018992,0.013093,0.082433,1.000000,0.011441,...,-0.003512,-0.003588,-0.013280,-0.011273,0.080949,0.045098,0.054440,-0.002352,0.002276,-0.002276
inq_last_6mths,-0.028376,0.232718,0.002401,-0.006142,0.037144,0.073945,-0.005728,0.023656,0.011441,1.000000,...,0.003103,0.029906,0.006662,0.011042,-0.004886,0.038563,0.025425,0.003774,0.002090,-0.002090


In [49]:
df_clean.corr().loan_status.sort_values(ascending=False)[1:]

int_rate                                0.176271
inq_last_6mths                          0.073945
grade_F                                 0.069778
grade_E                                 0.068427
grade_D                                 0.061359
revol_util                              0.051576
grade_G                                 0.044486
term_ 60 months                         0.040696
home_ownership_RENT                     0.037431
purpose_small_business                  0.037346
purpose_other                           0.016731
purpose_debt_consolidation              0.016717
installment                             0.016039
payment_to_income_ratio_individual      0.010962
purpose_moving                          0.010515
purpose_wedding                         0.008742
purpose_house                           0.008735
purpose_educational                     0.006669
purpose_medical                         0.006432
home_ownership_OTHER                    0.005679
application_type_IND

In [53]:
pkl_path= './data/loan_preprocessed.pkl'

with open(pkl_path, 'wb') as pklfile:
    pickle.dump(df_clean, pklfile)